# CSE 258 Assignment 1
**Ming Ki Toby Cheng**

In [1]:
import gzip
from collections import defaultdict
import scipy
import scipy.optimize
from sklearn import linear_model
import numpy
import random
import matplotlib.pyplot as plt

In [2]:
def readGz(path):
    for l in gzip.open(path, "rt"):
        yield eval(l)


def readCSV(path):
    f = gzip.open(path, "rt")
    f.readline()
    for l in f:
        yield l.strip().split(",")

In [3]:
users = []
books = []
ratings = []

In [4]:
for user, book, _ in readCSV("train_Interactions.csv.gz"):
    users.append(user)
    books.append(book)
    ratings.append(_)

In [5]:
#Splitting data into training and validation
users_train = users[:190000]
books_train = books[:190000]
ratings_train = ratings[:190000]
users_valid = users[190000:]
books_valid = books[190000:]
ratings_valid = ratings[190000:]

In [6]:
userCount = defaultdict(set)
allBooks = set()

for user, book, _ in readCSV("train_Interactions.csv.gz"):
    userCount[user].add(book)
    allBooks.add(book)

In [7]:
users_valid_new = users_valid[:]
books_valid_new = books_valid[:]
read_valid = [1] * len(users_valid)
read_valid_new = read_valid[:]

In [8]:
# Generating negative entries for validations randomly
random.seed(1583)

for users in users_valid:
    unread_books = allBooks.difference(userCount[users])
    unread_books_list = list(unread_books)
    A = unread_books_list[random.randint(0, len(unread_books_list) - 1)]
    users_valid_new.append(users)
    books_valid_new.append(A)
    read_valid_new.append(0)

In [9]:
users_train_new = users_train[:]
books_train_new = books_train[:]
read_train = [1] * len(users_train)
read_train_new = read_train[:]

In [10]:
random.seed(1342)

for users in users_train:
    unread_books = allBooks.difference(userCount[users])
    unread_books_list = list(unread_books)
    A = unread_books_list[random.randint(0, len(unread_books_list) - 1)]
    users_train_new.append(users)
    books_train_new.append(A)
    read_train_new.append(0)


In [11]:
validation_set = list(zip(users_valid_new, books_valid_new, read_valid_new))

In [12]:
training_set = list(zip(users_train, books_train, read_train))
training_set_neg = list(zip(users_train_new, books_train_new, read_train_new))

In [13]:
bookCount = defaultdict(int)
totalRead = 0

for user, book,_ in training_set:
    bookCount[book] += 1
    totalRead += 1

In [14]:
mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

In [ ]:
# Shuffling validation data
random.seed(1234)
random.shuffle(validation_set)

In [16]:
# Defining sets for users and books
usersPerBook = defaultdict(set)
booksPerUser = defaultdict(set)
ratingsPerUser = defaultdict(list)
for user, book, rating in training_set:
    usersPerBook[book].add(user)
    booksPerUser[user].add(book)

In [17]:
# Defining Jaccard similarity function
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    return (numer / denom)

In [46]:
# Defining separate Jaccard Prediction function
def predictionJaccardReg_2(u, b):
    similarities = []
    books = booksPerUser[u]
    users = usersPerBook[b]
    for user in users:
        books_other = booksPerUser[user]
        sim = Jaccard(books, books_other)
        similarities.append((sim, user))
    similarities.sort(reverse = True)

    if similarities[0][0] == 1.0 and len(similarities) > 1:
        return(similarities[1][0])
    elif len(similarities) == 0:
        return(0)
    else:
        return(similarities[0][0])

In [40]:
training_set_neg[379999]

('u40837736', 'b16562997', 0)

In [41]:
predictionJaccardReg_2('u40837736', 'b16562997')

0.03333333333333333

In [43]:
# Defining separate Jaccard Prediction function
def predictionJaccardReg(u, b):
    similarities = []
    books = booksPerUser[u]
    users = usersPerBook[b]
    for book in books:
        users_other = usersPerBook[book]
        sim = Jaccard(users, users_other)
        similarities.append((sim, book))
    similarities.sort(reverse = True)

    if similarities[0][0] == 1.0 and len(similarities) > 1:
        return(similarities[1][0])
    else:
        return(similarities[0][0])

In [20]:
# Defining Cosine similarity function
def Cosine(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1) * len(s2)
    if denom == 0:
        return(0)
    else:
        return(numer / denom)

In [21]:
# Defining separate Cosine Prediction function
def predictionCosineReg(u, b):
    cos_sim = []
    books = booksPerUser[u]
    users = usersPerBook[b]
    for book in books:
        users_other = usersPerBook[book]
        csim = Cosine(users, users_other)
        cos_sim.append((csim, book))
        
    cos_sim.sort(reverse = True)

    return(cos_sim[0][0])

In [22]:
# Defining book read count for popularity
bookPop = defaultdict(int)

for bPop, b in mostPopular:
    bookPop[b] = bPop


In [47]:
jaccard_sim_max = []
popular_num_train = []
read_train_reg = []
cosine_sim_max = []
jaccard_user = []
for user, book, read in training_set_neg:
    jaccard_sim_max.append(predictionJaccardReg(user,book))
    read_train_reg.append(read)
    popular_num_train.append(bookPop[book])
    cosine_sim_max.append(predictionCosineReg(user,book))
    jaccard_user.append(predictionJaccardReg_2(user,book))
    #if book in return1:
    #    predictions_reg.append(1)
    #else:
    #    predictions_reg.append(0)
    

IndexError: list index out of range

In [ ]:
jaccard_sim_val = []
popular_num_val = []
read_val_reg = []
cosine_val = []
jaccard_user_val = []

for user, book, _ in validation_set:
    jaccard_sim_val.append(predictionJaccardReg(user, book))
    read_val_reg.append(_)
    popular_num_val.append(bookPop[book])
    cosine_val.append(predictionCosineReg(user,book))
    jaccard_user_val.append(predictionJaccardReg_2(user,book))

    #if book in return1:
    #    predictions_val.append(1)
    #else:
    #    predictions_val.append(0)

In [ ]:
training_set_reg = list(zip(popular_num_train, jaccard_sim_max, cosine_sim_max, read_train_reg, jaccard_user))
validation_set_reg = list(zip(popular_num_val, jaccard_sim_val, cosine_val, read_val_reg, jaccard_user_val))

In [ ]:
# Shuffling training data
random.seed(1234)
random.shuffle(training_set_reg)

In [ ]:
def feature(datum):
    feat = [1, int(datum[0]), float(datum[1]), float(datum[2])] 
    return feat

In [ ]:
X_train = [feature(d) for d in training_set_reg]
Y_train = [d[2] for d in training_set_reg ]


In [ ]:
model = linear_model.LogisticRegression(C=0.01, class_weight = 'balanced')
model.fit(X_train, Y_train)

In [ ]:
X_valid = [feature(d) for d in validation_set_reg]
y_valid = [d[3] for d in validation_set_reg ]

In [ ]:
prediction = model.predict(X_train)
predictions = model.predict(X_valid)

In [ ]:
theta = model.coef_
theta

In [ ]:
X_train = [feature(d) for d in training_set_reg]
Y_train = [d[3] for d in training_set_reg ]

X_valid = [feature(d) for d in validation_set_reg]
y_valid = [d[3] for d in validation_set_reg ]

regularization_value = []
train_acc = []
train_BER = []
val_acc = []
val_BER = []
#numpy.arange(0.007, 0.009, 0.0001)

for i in numpy.arange(0.00001, 0.00003, 0.000001):
    model = linear_model.LogisticRegression(C=i)
    model.fit(X_train, Y_train)

    prediction = model.predict(X_train)
    predictions = model.predict(X_valid)

    TP_train = sum([(p and l) for (p, l) in zip(prediction, Y_train)])
    FP_train = sum([(p and not l) for (p, l) in zip(prediction, Y_train)])
    TN_train = sum([(not p and not l) for (p, l) in zip(prediction, Y_train)])
    FN_train = sum([(not p and l) for (p, l) in zip(prediction, Y_train)])
    accu_train = (TP_train + TN_train) / (TP_train + FP_train + TN_train + FN_train)

    TPR_train = TP_train / (TP_train + FN_train)
    TNR_train = TN_train / (TN_train + FP_train)
    BER_train = 1 - 0.5 * (TPR_train + TNR_train)

    TP_valid = sum([(p and l) for (p, l) in zip(predictions, y_valid)])
    FP_valid = sum([(p and not l) for (p, l) in zip(predictions, y_valid)])
    TN_valid = sum([(not p and not l) for (p, l) in zip(predictions, y_valid)])
    FN_valid = sum([(not p and l) for (p, l) in zip(predictions, y_valid)])
    accu_valid = (TP_valid + TN_valid) / (TP_valid + FP_valid + TN_valid + FN_valid)

    TPR_valid = TP_valid / (TP_valid + FN_valid)
    TNR_valid = TN_valid / (TN_valid + FP_valid)
    BER_valid = 1 - 0.5 * (TPR_valid + TNR_valid)

    regularization_value.append(i)
    train_acc.append(accu_train)
    train_BER.append(BER_train)
    val_acc.append(accu_valid)
    val_BER.append(BER_valid)

In [ ]:
plt.plot(regularization_value, train_acc, 'r--')
plt.ylabel('Accuracy')
plt.xlabel('Regularization Values')
plt.show()

In [ ]:
plt.plot(regularization_value, val_acc, 'b--')
plt.ylabel('Accuracy')
plt.xlabel('Regularization Values')
plt.show()

In [ ]:
plt.plot(regularization_value, val_BER, 'b--', regularization_value, train_BER, 'r--')
plt.ylabel('BER')
plt.xlabel('Regularization Values')
plt.show()

In [ ]:
print(val_acc.index(max(val_acc)), regularization_value[val_acc.index(max(val_acc))],  max(val_acc))

In [ ]:
print(val_BER.index(min(val_BER)), regularization_value[val_BER.index(min(val_BER))],  min(val_BER))

In [ ]:
X_train = [feature(d) for d in training_set_reg]
Y_train = [d[2] for d in training_set_reg ]

X_valid = [feature(d) for d in validation_set_reg]
y_valid = [d[2] for d in validation_set_reg ]

regularization_value = []
train_acc = []
train_BER = []
val_acc = []
val_BER = []

for i in numpy.arange(0.007, 0.009, 0.0001):
    model = linear_model.LogisticRegression(C=i)
    model.fit(X_train, Y_train)

    prediction = model.predict(X_train)
    predictions = model.predict(X_valid)

    TP_train = sum([(p and l) for (p, l) in zip(prediction, Y_train)])
    FP_train = sum([(p and not l) for (p, l) in zip(prediction, Y_train)])
    TN_train = sum([(not p and not l) for (p, l) in zip(prediction, Y_train)])
    FN_train = sum([(not p and l) for (p, l) in zip(prediction, Y_train)])
    accu_train = (TP_train + TN_train) / (TP_train + FP_train + TN_train + FN_train)

    TPR_train = TP_train / (TP_train + FN_train)
    TNR_train = TN_train / (TN_train + FP_train)
    BER_train = 1 - 0.5 * (TPR_train + TNR_train)

    TP_valid = sum([(p and l) for (p, l) in zip(predictions, y_valid)])
    FP_valid = sum([(p and not l) for (p, l) in zip(predictions, y_valid)])
    TN_valid = sum([(not p and not l) for (p, l) in zip(predictions, y_valid)])
    FN_valid = sum([(not p and l) for (p, l) in zip(predictions, y_valid)])
    accu_valid = (TP_valid + TN_valid) / (TP_valid + FP_valid + TN_valid + FN_valid)

    TPR_valid = TP_valid / (TP_valid + FN_valid)
    TNR_valid = TN_valid / (TN_valid + FP_valid)
    BER_valid = 1 - 0.5 * (TPR_valid + TNR_valid)

    regularization_value.append(i)
    train_acc.append(accu_train)
    train_BER.append(BER_train)
    val_acc.append(accu_valid)
    val_BER.append(BER_valid)

In [ ]:
plt.plot(regularization_value, train_acc, 'r--')
plt.ylabel('Accuracy')
plt.xlabel('Regularization Values')
plt.show()

In [ ]:
plt.plot(regularization_value, val_acc, 'b--')
plt.ylabel('Accuracy')
plt.xlabel('Regularization Values')
plt.show()

In [ ]:
print(val_acc.index(max(val_acc)), regularization_value[val_acc.index(max(val_acc))],  max(val_acc))

In [ ]:
plt.plot(regularization_value, train_BER, 'r--')
plt.ylabel('BER')
plt.xlabel('Regularization Values')
plt.show()

In [ ]:
plt.plot(regularization_value, val_BER, 'b--')
plt.ylabel('BER')
plt.xlabel('Regularization Values')
plt.show()

In [ ]:
print(val_BER.index(min(val_BER)), regularization_value[val_BER.index(min(val_BER))],  min(val_BER))

In [ ]:
model = linear_model.LogisticRegression(C=0.000023)
model.fit(X_train, Y_train)

In [ ]:
predictionCosineReg(user,book)

In [ ]:
# Writing predictions of test set to file
X_test = []
count = 0

predictions = open("predictions_Read_Assignment1.txt", "w")
for l in open("pairs_Read.txt"):
    if l.startswith("userID"):
        # header
        continue
    u, b = l.strip().split("-")
    jaccard_test=predictionJaccardReg(u, b)
    popular_test=bookPop[b]
    cosine_test =predictionCosineReg(u,b)
    X_test.append([1, popular_test, jaccard_test, cosine_test])

prediction_test = model.predict(X_test)
prediction_test = list(prediction_test)
for l in open("pairs_Read.txt"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u, b = l.strip().split("-")
    predictions.write(u + "-" + b + ','+str(prediction_test[count])+'\n')
    count+=1
predictions.close()

In [ ]:
# Writing predictions of test set to file
jaccard_test = []
popular_test = []
X_test = []
count = 0

predictions = open("predictions_Read_Assignment1.txt", "w")
for l in open("pairs_Read.txt"):
    if l.startswith("userID"):
        # header
        continue
    u, b = l.strip().split("-")
    jaccard_test=predictionJaccardReg(u, b)
    popular_test=bookPop[b]
    X_test.append([1, popular_test, jaccard_test])

prediction_test = model.predict(X_test)
prediction_test = list(prediction_test)
for l in open("pairs_Read.txt"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u, b = l.strip().split("-")
    predictions.write(u + "-" + b + ','+str(prediction_test[count])+'\n')
    count+=1
predictions.close()

In [ ]:
# Predictions, accuracy and BER based on combination of new popularity and Jaccard functions
predictions = []
y_valid = []

for users, books, read in validation_set:
    y_valid.append(read)
    if books in return1 and predictionJaccard(users, books) == 1:
        predictions.append(1)
    else:
        predictions.append(0)

TP_valid = sum([(p and l) for (p, l) in zip(predictions, y_valid)])
FP_valid = sum([(p and not l) for (p, l) in zip(predictions, y_valid)])
TN_valid = sum([(not p and not l) for (p, l) in zip(predictions, y_valid)])
FN_valid = sum([(not p and l) for (p, l) in zip(predictions, y_valid)])
accu_valid = (TP_valid + TN_valid) / (TP_valid + FP_valid + TN_valid + FN_valid)

TPR_valid = TP_valid / (TP_valid + FN_valid)
TNR_valid = TN_valid / (TN_valid + FP_valid)
BER_valid = 1 - 0.5 * (TPR_valid + TNR_valid)

## Accuracy and BER of Model on Set
print("Accuracy on validation set:", accu_valid)
print("BER on validation set:", BER_valid)

In [ ]:
# Predictions, accuracy and BER based on combination of new popularity and Jaccard functions
predictions = []
y_valid = []
Jacthres = []
weightthres = []
accur = []
BER = []

for num in numpy.arange(1, 3, 0.1):
    for jacnum in numpy.arange(0.001,0.01,0.001):
        predictions = []
        y_valid = []
        
        def predictionJaccard2(u, b):
            similarities = []
            books = booksPerUser[u]
            users = usersPerBook[b]
            for book in books:
                users_other = usersPerBook[book]
                sim = Jaccard(users, users_other)
                similarities.append((sim, book))
            if max(similarities)[0] > jacnum:
                return 1
            else:
                return 0

        return1 = set()
        count = 0
        for ic, i in mostPopular:
            count += ic
            return1.add(i)
            if count > totalRead / num:
                break

        for users, books, read in validation_set:
            y_valid.append(read)
            if books in return1 or predictionJaccard2(users, books) == 1:
                predictions.append(1)
            else:
                predictions.append(0)

        TP_valid = sum([(p and l) for (p, l) in zip(predictions, y_valid)])
        FP_valid = sum([(p and not l) for (p, l) in zip(predictions, y_valid)])
        TN_valid = sum([(not p and not l) for (p, l) in zip(predictions, y_valid)])
        FN_valid = sum([(not p and l) for (p, l) in zip(predictions, y_valid)])
        accu_valid = (TP_valid + TN_valid) / (TP_valid + FP_valid + TN_valid + FN_valid)

        TPR_valid = TP_valid / (TP_valid + FN_valid)
        TNR_valid = TN_valid / (TN_valid + FP_valid)
        BER_valid = 1 - 0.5 * (TPR_valid + TNR_valid)
        
        Jacthres.append(jacnum)
        weightthres.append(num)
        accur.append(accu_valid)
        BER.append(BER_valid)
        ## Accuracy and BER of Model on Set
        #print('Jaccard threshold', jacnum)
        #print('Most popular weight', num)
        #print("Accuracy on validation set:", accu_valid)
        #print("BER on validation set:", BER_valid)

In [ ]:
print(Jacthres[179])
print(weightthres[179])
print(max(accur))
print(accur.index(max(accur)))
print(min(BER))

In [ ]:
books in return1 and predictionJaccard(users, books) == 1:
max(similarities)[0] > 0.008:
totalRead / 1.6
Accuracy on validation set: 0.66125
BER on validation set: 0.33875

**Kaggle Username: tobycheng or Toby Cheng**

## Part 2

### Question 9


In [ ]:
users = []
books = []
ratings = []
for user, book, _ in readCSV("train_Interactions.csv.gz"):
    users.append(user)
    books.append(book)
    ratings.append(int(_))

In [ ]:
# Re-initializing data
users_train = users[:190000]
books_train = books[:190000]
ratings_train = ratings[:190000]
users_valid = users[190000:]
books_valid = books[190000:]
ratings_valid = ratings[190000:]

In [ ]:
training_rating = list(zip(users_train, books_train, ratings_train))
validation_rating = list(zip(users_valid, books_valid, ratings_valid))
total_set = list(zip(users, books, ratings))

In [ ]:
# Forming dictionary with user and book combinations
ratingPerCombo = {}
for user, book,r in total_set:
    usersPerBook[book].add(user)
    booksPerUser[user].add(book)
    ratingPerCombo[(user,book)] = int(r)

In [ ]:
# Initializing all user ratings and all books dictionaries
allRatings = []
userRatings = defaultdict(list)
bookRatings = defaultdict(list)

# All ratings for each user and each book
for user, book, r in total_set:
    r = int(r)
    allRatings.append(r)
    userRatings[user].append(r)
    bookRatings[book].append(r)

globalAverage = sum(allRatings) / len(allRatings)
userAverage = {}
userBias = {}
user_total = 0
bookAverage = {}
bookBias = {}
book_total = 0

# Initializing user and book biases
for user in userRatings:
    userAverage[user] = sum(userRatings[user]) / len(userRatings[user])
    
for user in userAverage:
    user_total += float(userAverage[user])
    
for user in userAverage:
    userBias[user] = userAverage[user] - (user_total/len(userAverage))

for book in bookRatings:
    bookAverage[book] = sum(bookRatings[book]) / len(bookRatings[book])
    
for book in bookAverage:
    book_total += float(bookAverage[book])
    
for book in bookAverage:
    bookBias[book] = bookAverage[book] - (book_total/len(bookAverage))

In [ ]:
# Defining predictor function
def prediction(user, book):
    if user not in userBias and book in bookBias:
        return alpha + bookBias[book]
    if user in userBias and book not in bookBias:
        return alpha + userBias[user]
    if user not in userBias and book not in bookBias:
        return alpha
    if user in userBias and book in bookBias:
        return alpha + userBias[user] + bookBias[book]

In [ ]:
# Defining MSE function
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [ ]:
# Defining alpha, bookBias, userBias by convergence from Training data and lambda value 1
lamb = 1
alpha_sum = 0
alpha = 0
MSE_diff = 5
trial = 0

while MSE_diff > 0.00005 or trial > 1000:
    model_predictions = []
    alpha_sum = 0
    
    for user, book, r in training_rating:
        alpha_sum += (r-(userBias[user] + bookBias[book]))
    alpha = alpha_sum/len(training_rating)
    
    for user in booksPerUser:
        beta_U_Sum = 0
        for books in booksPerUser[user]:
            beta_U_Sum += ratingPerCombo[user,books] - (alpha + bookBias[books])
        userBias[user] = beta_U_Sum/(lamb+ len(booksPerUser[user]))
    
    for book in usersPerBook:
        beta_I_Sum = 0
        for users in usersPerBook[book]:
            beta_I_Sum += ratingPerCombo[users,book] - (alpha + userBias[users])
        bookBias[book] = beta_I_Sum/(lamb+ len(usersPerBook[book]))
    
    for user, book, r in training_rating:
        model_predictions.append(prediction(user, book))
    
    if trial == 0:
        MSE_0 = 0

    MSE_1 = MSE(model_predictions, ratings_train)
    
    MSE_diff = abs(MSE_1 - MSE_0)

    print('MSE_new:', MSE_1)
    
    MSE_0 = MSE_1
    trial += 1 

In [ ]:
# Predictions and MSE on validation data
model_predictions = []
for user, book, r in validation_rating:
    model_predictions.append(prediction(user, book))

MSE_valid = MSE(model_predictions, ratings_valid)
print('MSE on validation set:',MSE_valid)

### Question 10

In [ ]:
# User and Book with lowest user bias value
minimum = min(userBias, key=userBias.get)  
print('User with smallest user bias value:', minimum)
print('Smallest user bias value:', userBias[minimum])

In [ ]:
# User and Book with highest user bias value
maximum = max(userBias, key=userBias.get)  
print('User with largest user bias value:', maximum)
print('Largest user bias value:', userBias[maximum])

In [ ]:
# User and Book with lowest book bias value
minimum = min(bookBias, key=bookBias.get)  

print('Book with smallest book bias value:', minimum)
print('Smallest book bias value:', bookBias[minimum])

In [ ]:
# User and Book with highest book bias value
maximum = max(bookBias, key=bookBias.get)  

print('Book with largest book bias value:', maximum)
print('Largest book bias value:', bookBias[maximum])

### Question 11

In [ ]:
# Initializing user and book biases
allRatings = []
userRatings = defaultdict(list)
bookRatings = defaultdict(list)

for user, book, r in total_set:
    r = int(r)
    allRatings.append(r)
    userRatings[user].append(r)
    bookRatings[book].append(r)

globalAverage = sum(allRatings) / len(allRatings)
userAverage = {}
userBias = {}
user_total = 0
bookAverage = {}
bookBias = {}
book_total = 0

# Initializing user and book biases
for user in userRatings:
    userAverage[user] = sum(userRatings[user]) / len(userRatings[user])
    
for user in userAverage:
    user_total += float(userAverage[user])
    
for user in userAverage:
    userBias[user] = userAverage[user] - (user_total/len(userAverage))

for book in bookRatings:
    bookAverage[book] = sum(bookRatings[book]) / len(bookRatings[book])
    
for book in bookAverage:
    book_total += float(bookAverage[book])
    
for book in bookAverage:
    bookBias[book] = bookAverage[book] - (book_total/len(bookAverage))
    
# Defining alpha, bookBias, userBias by convergence from Training data and lambda value 1
lambda_value = []
MSE_list = []
MSE_valid_list = []

for lambi in numpy.arange(0, 1, 0.01):
    lamb = lambi
    alpha_sum = 0
    alpha = 0
    MSE_diff = 5
    trial = 0

    
    print(lamb)
    
    while MSE_diff > 0.00005 or trial > 1000:
        model_predictions = []
        alpha_sum = 0

        for user, book, r in training_rating:
            alpha_sum += (r-(userBias[user] + bookBias[book]))
        alpha = alpha_sum/len(training_rating)

        for user in booksPerUser:
            beta_U_Sum = 0
            for books in booksPerUser[user]:
                beta_U_Sum += ratingPerCombo[user,books] - (alpha + bookBias[books])
            userBias[user] = beta_U_Sum/(lamb+ len(booksPerUser[user]))

        for book in usersPerBook:
            beta_I_Sum = 0
            for users in usersPerBook[book]:
                beta_I_Sum += ratingPerCombo[users,book] - (alpha + userBias[users])
            bookBias[book] = beta_I_Sum/(lamb+ len(usersPerBook[book]))

        for user, book, r in training_rating:
            model_predictions.append(prediction(user, book))

        if trial == 0:
            MSE_0 = 0

        MSE_1 = MSE(model_predictions, ratings_train)

        MSE_diff = abs(MSE_1 - MSE_0)

        #print('MSE_new:', MSE_1)

        MSE_0 = MSE_1
        trial += 1 
    
    lambda_value.append(lambi)
    MSE_list.append(MSE_1)
    model_predictions = []
    for user, book, r in validation_rating:
        model_predictions.append(prediction(user, book))

    MSE_valid = MSE(model_predictions, ratings_valid)
    MSE_valid_list.append(MSE_valid)
    print(MSE_valid)

In [ ]:
plt.plot(lambda_value, MSE_valid_list)

In [ ]:
print(MSE_list.index(min(MSE_list)))
print(MSE_list[10])
print(lambda_value[10])


In [ ]:
lamb = 0.001
alpha_sum = 0
alpha = 0
MSE_diff = 5
trial = 0

while MSE_diff > 0.00005 or trial > 1000:
    model_predictions = []
    alpha_sum = 0

    for user, book, r in training_rating:
        alpha_sum += (r-(userBias[user] + bookBias[book]))
    alpha = alpha_sum/len(training_rating)

    for user in booksPerUser:
        beta_U_Sum = 0
        for books in booksPerUser[user]:
            beta_U_Sum += ratingPerCombo[user,books] - (alpha + bookBias[books])
        userBias[user] = beta_U_Sum/(lamb+ len(booksPerUser[user]))

    for book in usersPerBook:
        beta_I_Sum = 0
        for users in usersPerBook[book]:
            beta_I_Sum += ratingPerCombo[users,book] - (alpha + userBias[users])
        bookBias[book] = beta_I_Sum/(lamb+ len(usersPerBook[book]))

    for user, book, r in training_rating:
        model_predictions.append(prediction(user, book))

    if trial == 0:
        MSE_0 = 0

    MSE_1 = MSE(model_predictions, ratings_train)

    MSE_diff = abs(MSE_1 - MSE_0)

    print('MSE_new:', MSE_1)

    MSE_0 = MSE_1
    trial += 1 


In [ ]:
# Predictions and MSE on validation data
model_predictions = []
for user, book, r in validation_rating:
    model_predictions.append(prediction(user, book))

MSE_valid = MSE(model_predictions, ratings_valid)
print(MSE_valid)

After many trials found that lambda value of 3 seems to have the lowest validation MSE at 1.1080896979035464

In [ ]:
# Writing predictions of test set to file
predictions = open("predictions_Rating_Assignment1.txt", "w")
for l in open("pairs_Rating.txt"):
    if l.startswith("userID"):
        # header
        predictions.write(l)
        continue
    u, b = l.strip().split("-")
    
    predictions.write(u + "-" + b + "," + str(prediction(u,b)) + "\n")

predictions.close()

**Kaggle Username: tobycheng or Toby Cheng**